In [ ]:
pip install yfinance


In [ ]:
import yfinance as yf
import pandas as pd

def fetch_stock_data(ticker, period='1y', interval='1d'):
    stock_data = yf.download(ticker, period=period, interval=interval)
    stock_data.reset_index(inplace=True)
    return stock_data

# Example usage
ticker = 'AAPL'
stock_data = fetch_stock_data(ticker)
print(stock_data.head())


In [ ]:
pip install ta


In [ ]:
import ta

def add_technical_indicators(df):
    df['SMA'] = ta.trend.sma_indicator(df['Close'], window=20)
    df['EMA'] = ta.trend.ema_indicator(df['Close'], window=20)
    df['RSI'] = ta.momentum.rsi(df['Close'], window=14)
    df['MACD'] = ta.trend.macd_diff(df['Close'])
    return df

# Example usage
stock_data = add_technical_indicators(stock_data)
print(stock_data[['Close', 'SMA', 'EMA', 'RSI', 'MACD']].head())


In [ ]:
def calculate_sma(df, window):
    return df['Close'].rolling(window=window).mean()

# Example usage
stock_data['SMA'] = calculate_sma(stock_data, window=20)
print(stock_data['SMA'])


In [ ]:
def calculate_ema(df, window):
    return df['Close'].ewm(span=window, adjust=False).mean()

# Example usage
stock_data['EMA'] = calculate_ema(stock_data, window=20)

print(stock_data['EMA'])


In [ ]:
def calculate_rsi(df, window):
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Example usage
stock_data['RSI'] = calculate_rsi(stock_data, window=14)
print(stock_data['RSI'])


In [ ]:
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    short_ema = df['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = df['Close'].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    macd_diff = macd - signal
    return macd, signal, macd_diff

# Example usage
stock_data['MACD'], stock_data['Signal Line'], stock_data['MACD Histogram'] = calculate_macd(stock_data)
print(stock_data['MACD'])
print(stock_data['MACD Histogram'])
print(stock_data['Signal Line'])


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

def fetch_stock_data(ticker, period='1y', interval='1d'):
    stock_data = yf.download(ticker, period=period, interval=interval)
    stock_data.reset_index(inplace=True)
    return stock_data

def calculate_sma(df, window):
    return df['Close'].rolling(window=window).mean()

def calculate_ema(df, window):
    return df['Close'].ewm(span=window, adjust=False).mean()

def calculate_rsi(df, window):
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    short_ema = df['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = df['Close'].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    macd_diff = macd - signal
    return macd, signal, macd_diff

def add_technical_indicators(df):
    df['SMA'] = calculate_sma(df, window=20)
    df['EMA'] = calculate_ema(df, window=20)
    df['RSI'] = calculate_rsi(df, window=14)
    df['MACD'], df['Signal Line'], df['MACD Histogram'] = calculate_macd(df)
    return df


# Plotting the stock data with technical indicators
def plot_stock_data(df, ticker):
    plt.figure(figsize=(14, 10))

    # Plot Close price, SMA, and EMA
    plt.subplot(3, 1, 1)
    plt.plot(df['Date'], df['Close'], label='Close Price', color='blue')
    plt.plot(df['Date'], df['SMA'], label='SMA', color='orange')
    plt.plot(df['Date'], df['EMA'], label='EMA', color='green')
    plt.title(f'{ticker} Stock Price with SMA and EMA')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()

    # Plot RSI
    plt.subplot(3, 1, 2)
    plt.plot(df['Date'], df['RSI'], label='RSI', color='purple')
    plt.axhline(70, color='red', linestyle='--', linewidth=0.5)
    plt.axhline(30, color='green', linestyle='--', linewidth=0.5)
    plt.title('Relative Strength Index (RSI)')
    plt.xlabel('Date')
    plt.ylabel('RSI')
    plt.legend()

    # Plot MACD and Signal Line
    plt.subplot(3, 1, 3)
    plt.plot(df['Date'], df['MACD'], label='MACD', color='blue')
    plt.plot(df['Date'], df['Signal Line'], label='Signal Line', color='orange')
    plt.fill_between(df['Date'], df['MACD Histogram'], color='gray', alpha=0.5, label='MACD Histogram')
    plt.title('Moving Average Convergence Divergence (MACD)')
    plt.xlabel('Date')
    plt.ylabel('MACD')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Fetch and analyze stock data
ticker = 'AAPL'
stock_data = fetch_stock_data(ticker)
stock_data = add_technical_indicators(stock_data)

# Plot stock data
plot_stock_data(stock_data, ticker)

def prepare_data(df):
    df = df.dropna()  # Drop rows with NaN values
    X = df[['SMA', 'EMA', 'RSI', 'MACD']]
    y = df['Close']
    return X, y

# Train and evaluate the regression model
def train_regression_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error: {mse}')
    return model

# Generate buy/sell signals based on predictions
def generate_signals(df, model):
    df = df.dropna()  # Drop rows with NaN values
    X = df[['SMA', 'EMA', 'RSI', 'MACD']]
    df['Predicted Close'] = model.predict(X)
    df['Signal'] = 0
    df['Signal'][1:] = np.where(df['Predicted Close'][1:] > df['Close'].shift(1)[1:], 1, -1)
    df['Position'] = df['Signal'].diff()
    return df


# Plot stock data
plot_stock_data(stock_data, ticker)

# Fetch and analyze stock data
ticker = 'AAPL'
stock_data = fetch_stock_data(ticker)
stock_data = add_technical_indicators(stock_data)

# Display the data with technical indicators
print(stock_data[['Date', 'Close', 'SMA', 'EMA', 'RSI', 'MACD', 'Signal Line']].tail())


In [ ]:



import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Fetch historical stock data
def fetch_stock_data(ticker, period='1y', interval='1d'):
    stock_data = yf.download(ticker, period=period, interval=interval)
    stock_data.reset_index(inplace=True)
    return stock_data

# Calculate Simple Moving Average (SMA)
def calculate_sma(df, window):
    return df['Close'].rolling(window=window).mean()

# Calculate Exponential Moving Average (EMA)
def calculate_ema(df, window):
    return df['Close'].ewm(span=window, adjust=False).mean()

# Calculate Relative Strength Index (RSI)
def calculate_rsi(df, window):
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Calculate Moving Average Convergence Divergence (MACD)
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    short_ema = df['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = df['Close'].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    macd_diff = macd - signal
    return macd, signal, macd_diff

# Add technical indicators to the dataframe
def add_technical_indicators(df):
    df['SMA'] = calculate_sma(df, window=20)
    df['EMA'] = calculate_ema(df, window=20)
    df['RSI'] = calculate_rsi(df, window=14)
    df['MACD'], df['Signal Line'], df['MACD Histogram'] = calculate_macd(df)
    return df

# Plotting the stock data with technical indicators
def plot_stock_data(df, ticker):
    plt.figure(figsize=(14, 10))

    # Plot Close price, SMA, and EMA
    plt.subplot(3, 1, 1)
    plt.plot(df['Date'], df['Close'], label='Close Price', color='blue')
    plt.plot(df['Date'], df['SMA'], label='SMA', color='orange')
    plt.plot(df['Date'], df['EMA'], label='EMA', color='green')
    plt.title(f'{ticker} Stock Price with SMA and EMA')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()

    # Plot RSI
    plt.subplot(3, 1, 2)
    plt.plot(df['Date'], df['RSI'], label='RSI', color='purple')
    plt.axhline(70, color='red', linestyle='--', linewidth=0.5)
    plt.axhline(30, color='green', linestyle='--', linewidth=0.5)
    plt.title('Relative Strength Index (RSI)')
    plt.xlabel('Date')
    plt.ylabel('RSI')
    plt.legend()

    # Plot MACD and Signal Line
    plt.subplot(3, 1, 3)
    plt.plot(df['Date'], df['MACD'], label='MACD', color='blue')
    plt.plot(df['Date'], df['Signal Line'], label='Signal Line', color='orange')
    plt.fill_between(df['Date'], df['MACD Histogram'], color='gray', alpha=0.5, label='MACD Histogram')
    plt.title('Moving Average Convergence Divergence (MACD)')
    plt.xlabel('Date')
    plt.ylabel('MACD')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Prepare the data for regression
def prepare_data(df):
    df = df.dropna()  # Drop rows with NaN values
    X = df[['SMA', 'EMA', 'RSI', 'MACD']]
    y = df['Close']
    return X, y

# Train and evaluate the regression model
def train_regression_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error: {mse}')
    return model

# Generate buy/sell signals based on predictions
def generate_signals(df, model):
    df = df.dropna()  # Drop rows with NaN values
    X = df[['SMA', 'EMA', 'RSI', 'MACD']]
    df['Predicted Close'] = model.predict(X)
    df['Signal'] = 0
    df['Signal'][1:] = np.where(df['Predicted Close'][1:] > df['Close'].shift(1)[1:], 1, -1)
    df['Position'] = df['Signal'].diff()
    return df

# Fetch and analyze stock data
ticker = 'AAPL'
stock_data = fetch_stock_data(ticker)
stock_data = add_technical_indicators(stock_data)
X, y = prepare_data(stock_data)
model = train_regression_model(X, y)
stock_data = generate_signals(stock_data, model)

# Plot stock data with technical indicators
plot_stock_data(stock_data, ticker)

# Display the latest signals
print(stock_data[['Date', 'Close', 'SMA', 'EMA', 'RSI', 'MACD', 'Predicted Close', 'Signal', 'Position']].tail())


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Fetch historical stock data
def fetch_stock_data(ticker, period='1y', interval='1d'):
    stock_data = yf.download(ticker, period=period, interval=interval)
    stock_data.reset_index(inplace=True)
    return stock_data

# Calculate Simple Moving Average (SMA)
def calculate_sma(df, window):
    return df['Close'].rolling(window=window).mean()

# Calculate Exponential Moving Average (EMA)
def calculate_ema(df, window):
    return df['Close'].ewm(span=window, adjust=False).mean()

# Calculate Relative Strength Index (RSI)
def calculate_rsi(df, window):
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Calculate Moving Average Convergence Divergence (MACD)
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    short_ema = df['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = df['Close'].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    macd_diff = macd - signal
    return macd, signal, macd_diff

# Add technical indicators to the dataframe
def add_technical_indicators(df):
    df['SMA'] = calculate_sma(df, window=20)
    df['EMA'] = calculate_ema(df, window=20)
    df['RSI'] = calculate_rsi(df, window=14)
    df['MACD'], df['Signal Line'], df['MACD Histogram'] = calculate_macd(df)
    return df

# Plotting the stock data with technical indicators
def plot_stock_data(df, ticker):
    plt.figure(figsize=(14, 10))

    # Plot Close price, SMA, and EMA
    plt.subplot(3, 1, 1)
    plt.plot(df['Date'], df['Close'], label='Close Price', color='blue')
    plt.plot(df['Date'], df['SMA'], label='SMA', color='orange')
    plt.plot(df['Date'], df['EMA'], label='EMA', color='green')
    plt.title(f'{ticker} Stock Price with SMA and EMA')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()

    # Plot RSI
    plt.subplot(3, 1, 2)
    plt.plot(df['Date'], df['RSI'], label='RSI', color='purple')
    plt.axhline(70, color='red', linestyle='--', linewidth=0.5)
    plt.axhline(30, color='green', linestyle='--', linewidth=0.5)
    plt.title('Relative Strength Index (RSI)')
    plt.xlabel('Date')
    plt.ylabel('RSI')
    plt.legend()

    # Plot MACD and Signal Line
    plt.subplot(3, 1, 3)
    plt.plot(df['Date'], df['MACD'], label='MACD', color='blue')
    plt.plot(df['Date'], df['Signal Line'], label='Signal Line', color='orange')
    plt.fill_between(df['Date'], df['MACD Histogram'], color='gray', alpha=0.5, label='MACD Histogram')
    plt.title('Moving Average Convergence Divergence (MACD)')
    plt.xlabel('Date')
    plt.ylabel('MACD')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Prepare the data for regression
def prepare_data(df):
    df = df.dropna()  # Drop rows with NaN values
    X = df[['SMA', 'EMA', 'RSI', 'MACD']]
    y = df['Close']
    return X, y

# Train and evaluate the regression model using Randomized Search
def train_regression_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define the model and parameter grid
    model = Ridge()
    param_distributions = {
        'alpha': np.logspace(-4, 4, 50)
    }

    # Perform Randomized Search
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=50, scoring='neg_mean_squared_error', cv=5, random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)

    # Best model
    best_model = random_search.best_estimator_
    print(f'Best Parameters: {random_search.best_params_}')

    # Evaluate the model
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error: {mse}')
    return best_model

# Generate buy/sell signals based on predictions
def generate_signals(df, model):
    df = df.dropna()  # Drop rows with NaN values
    X = df[['SMA', 'EMA', 'RSI', 'MACD']]
    df['Predicted Close'] = model.predict(X)
    df['Signal'] = 0
    df['Signal'][1:] = np.where(df['Predicted Close'][1:] > df['Close'].shift(1)[1:], 1, -1)
    df['Position'] = df['Signal'].diff()
    return df

# Fetch and analyze stock data
ticker = 'AAPL'
stock_data = fetch_stock_data(ticker)
stock_data = add_technical_indicators(stock_data)
X, y = prepare_data(stock_data)
model = train_regression_model(X, y)
stock_data = generate_signals(stock_data, model)

# Plot stock data with technical indicators
plot_stock_data(stock_data, ticker)

# Display the latest signals
print(stock_data[['Date', 'Close', 'SMA', 'EMA', 'RSI', 'MACD', 'Predicted Close', 'Signal', 'Position']].tail())
